# Final

The final project will consist of a comparison between several CNN architectures for tumor detection. The goal is both to create a high-performing algorithm for differentiating kidneys with tumor from those that are normal, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 31.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about the KiTS Challenge can be found here: https://kits21.kits-challenge.org/. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

[ 2022-06-08 18:04:49 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

Since the algorithms below may require slightly different model inputs, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

### Data Generator

To accomodate these various permutations, consider the following custom code to implement a nested generator strategy:

In [4]:
def G(gen, dims=2, task='cls', binarize=True):
    """
    Custom generator to modify raw labels for 2D/3D classification or segmentation tasks
    
    :params
    
      (generator) gen      : original unmodified generator
      (int)       dims     : 2D or 3D model
      (str)       task     : 'cls' or 'seg' 
      (bool)      binarize : whether or not to binarize original 3-class labels
    
    """
    assert task in ['cls', 'seg']

    for xs, _ in gen:

        # --- Convert segmentation into classification labels
        if task == 'cls':
            axis = (2, 3, 4) if dims == 2 else (1, 2, 3, 4)
            xs['lbl'] = np.max(xs['lbl'], axis=axis, keepdims=True)
            
        # --- Binarize
        if binarize:
            xs['lbl'] = xs['lbl'] == 2

        yield xs

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. Classification

The first task is to create any classification model for binary tumor detection. A 2D model will predict tumor vs. no tumor on a slice-by-slice basis whereas a 3D model will predict tumor vs. no tumor on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. For those that choose a 2D model, a reduction strategy must be implemented (see details further below).

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [5]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [6]:
kernel_size = (1, 3, 3) 
strides = (1, 2, 2)
    
    # --- Define kwargs
kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block componentsconv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=strides)))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=strides)))

concat = lambda a, b : layers.Concatenate()([a, b])
                                     
x = Input(shape=(None, 96, 96, 1), dtype='float32')

# --- Define projection
bneck = lambda filters, x : layers.Conv3D(
    filters=filters, 
    strides=1, 
    kernel_size=(1, 1, 1),
    padding='same')(relu(norm(x)))
l0 = conv(x, filters=16, strides=1)
# --- Define standard conv-conv block
l1 = conv1(32, l0)
l2 = conv1(32, l1)

# --- Define bottleneck conv-conv block
l1 = conv1(32, l0)
l2 = bneck(32, conv1(8, bneck(8, l1)))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(80, conv2(80, l5))
n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)

# --- Create logits
logits = layers.Conv3D(filters=2, kernel_size=1)(f0)

# --- Create backbone model
backbone = Model(inputs=x, outputs=logits)




In [7]:
inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 1, 1, 1), name='lbl')}    
logits = backbone(inputs['dat'])

# --- Define loss object
sce = losses.SparseCategoricalCrossentropy(from_logits=True)
# --- Create loss tensor
loss = sce(y_true=inputs['lbl'], y_pred=logits)
# --- Define accuracy
acc = metrics.sparse_categorical_accuracy(y_true=inputs['lbl'], y_pred=logits)
# --- Create training model
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

### Compile and train model

In [8]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []

for x in test_valid:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())

# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)
# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(acc, name='acc')

# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Compile model
training.compile(optimizer=optimizer)

# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh
# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='cls'),
    validation_data=G(gen_valid, dims=2, task='cls'),
    steps_per_epoch=100, 
    epochs=10,
    validation_steps=100,
    validation_freq=5)

[ 2022-06-08 18:05:32 ] [====================] 100.000% : Iterating | 000081    Acc: 0.6173
Sen: 0.6170
Spe: 0.6176
PPV: 0.6905
NPV: 0.5385
Epoch 1/10
100/100 [==============================] - 15s 118ms/step - loss: 0.7042 - acc: 0.6006
Epoch 2/10
100/100 [==============================] - 12s 118ms/step - loss: 0.5780 - acc: 0.7150
Epoch 3/10
100/100 [==============================] - 12s 118ms/step - loss: 0.5174 - acc: 0.7525
Epoch 4/10
100/100 [==============================] - 12s 124ms/step - loss: 0.4830 - acc: 0.7700
Epoch 5/10
100/100 [==============================] - 24s 239ms/step - loss: 0.4606 - acc: 0.7794 - val_loss: 0.8029 - val_acc: 0.6581
Epoch 6/10
100/100 [==============================] - 12s 120ms/step - loss: 0.4228 - acc: 0.8019
Epoch 7/10
100/100 [==============================] - 12s 119ms/step - loss: 0.4193 - acc: 0.8194
Epoch 8/10
100/100 [==============================] - 12s 124ms/step - loss: 0.3324 - acc: 0.8619
Epoch 9/10
100/100 [===================

In [9]:
# --- Serialize a model
backbone.save('./model1.hdf5')

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []


for x in test_train:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())
# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)

# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh

# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-08 02:39:29 ] [====================] 100.000% : Iterating | 000321    Acc: 0.8536
Sen: 0.8679
Spe: 0.8395
PPV: 0.8415
NPV: 0.8662


## 2. Segmentation

The second task is to create any segmentation model for binary tumor localization. A 2D model will predict tumor segmentation masks on a slice-by-slice basis whereas a 3D model will predict tumor segmentation masks on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. To do so, a reduction strategy must be implemented (see details further below).

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [ ]:
# --- Define model input 
x = Input(shape=(None, 96, 96, 1), dtype='float32')
# --- Define kwargs dictionary
kernel_size = (1, 3, 3) 
strides = (1, 2, 2)
    
    # --- Define kwargs
kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block componentsconv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)


# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))# --- Define single transpose
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))
# --- Define contracting layers
l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6  = tran2(48, l5)

concat = lambda a, b : layers.Concatenate()([a, b])
concat(l4, l6)

# --- Define expanding layers
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)


# --- Create logits
logits = layers.Conv3D(filters=2, **kwargs)(l10)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [ ]:
# --- Create training model
inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 96, 96, 1), name='lbl')}
# --- Define first step of new wrapper model
logits = backbone(inputs['dat'])
# --- Define loss object
sce = losses.SparseCategoricalCrossentropy(from_logits=True)

# --- Create loss tensor
loss = sce(y_true=inputs['lbl'], y_pred=logits)
def create_weights(lbl, pos_weight=5.0):
    
    # --- Create wgt
    wgt = tf.ones_like(lbl, dtype='float32')
    wgt = wgt + tf.cast(lbl == 2, dtype='float32') * (pos_weight - 1.0)
    
    return wgt
# --- Create weights
wgt = create_weights(inputs['lbl'])
def calculate_dsc(y_true, y_pred, weights=None, c=1):
    """
    Method to calculate the Dice score coefficient for given class

    :params

      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on

    """    
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 

    if weights is not None:
        true = true & (weights[..., 0] != 0)
        pred = pred & (weights[..., 0] != 0)

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))
dsc = calculate_dsc(y_true=inputs['lbl'], y_pred=logits)
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'dsc': dsc})
# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(dsc, name='dsc')

### Compile and train model

In [ ]:
# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Compile model
training.compile(optimizer=optimizer)



In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='seg')
test_valid = G(test_valid, dims=2, task='seg')

preds = []
trues = []

for x in test_valid:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())

# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)
# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh
# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh
# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)



# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='seg'),
    validation_data=G(gen_valid, dims=2, task='seg'),
    steps_per_epoch=100, 
    epochs=20,
    validation_steps=100,
    validation_freq=5)

[ 2022-06-08 02:47:30 ] [====================] 100.000% : Iterating | 000081    Epoch 1/20
100/100 [==============================] - 12s 120ms/step - loss: 0.0201 - dsc: 0.8040
Epoch 2/20
100/100 [==============================] - 12s 122ms/step - loss: 0.0214 - dsc: 0.8015
Epoch 3/20
100/100 [==============================] - 12s 124ms/step - loss: 0.0183 - dsc: 0.8288
Epoch 4/20
100/100 [==============================] - 12s 124ms/step - loss: 0.0179 - dsc: 0.8450
Epoch 5/20
100/100 [==============================] - 24s 242ms/step - loss: 0.0173 - dsc: 0.8460 - val_loss: 0.0810 - val_dsc: 0.4128
Epoch 6/20
100/100 [==============================] - 13s 126ms/step - loss: 0.0163 - dsc: 0.8444
Epoch 7/20
100/100 [==============================] - 13s 130ms/step - loss: 0.0148 - dsc: 0.8555
Epoch 8/20
100/100 [==============================] - 12s 125ms/step - loss: 0.0150 - dsc: 0.8651
Epoch 9/20
100/100 [==============================] - 12s 124ms/step - loss: 0.0139 - dsc: 0.8738
E

In [ ]:
# --- Serialize a model
backbone.save('./model2.hdf5')

In [ ]:
print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

Acc: 0.5802
Sen: 0.5745
Spe: 0.5882
PPV: 0.6585
NPV: 0.5000


In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='seg')
test_valid = G(test_valid, dims=2, task='seg')

preds = []
trues = []


for x in test_train:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())
# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)

# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh

# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-08 02:56:01 ] [====================] 100.000% : Iterating | 000321    Acc: 0.9626
Sen: 0.9686
Spe: 0.9568
PPV: 0.9565
NPV: 0.9688


## 3. Custom architecture

Finally, using any of the customizations described in class, find a top-performing model that may potentially yield some incremental benefit over the two baseline models above.

### Create generators

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)


### Define model

In [ ]:
# --- Define model input 
x = Input(shape=(None, 96, 96, 1), dtype='float32')
# --- Define kwargs dictionary
kernel_size = (1, 3, 3) 
strides = (1, 2, 2)
    
    # --- Define kwargs
kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}
# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

# --- Define model
l1 = conv1(32,  x)
# --- Squeeze (global pool)
sqz = layers.AveragePooling3D((1, l1.shape[2], l1.shape[3]))(l1)

# --- Excitation (reduce channels to 1 / R) ==> in this example set R = 4 arbitrarily
cha = int(l1.shape[-1] / 4)
exc = layers.Conv3D(filters=cha, kernel_size=1, activation='relu')(sqz)

# --- Scale (expand channels to original size)
sca = layers.Conv3D(filters=l1.shape[-1], kernel_size=1, activation='sigmoid')(exc)

# --- Modify l1
l1 = l1 * sca
# --- Define model
l1 = conv1(32,  x)
l2 = conv1(48,  conv2(48,  l1))
l3 = conv1(64,  conv2(64,  l2))
l4 = conv1(80,  conv2(80,  l3))
l5 = conv1(96,  conv2(96,  l4))
l6 = conv1(128, conv2(128, l5))
# --- Extract shape and reshape
n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)

# --- Create logits
logits = layers.Conv3D(filters=2, kernel_size=1)(f0)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [ ]:
inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 96, 96, 1), name='lbl')}
# --- Define first step of new wrapper model
logits = backbone(inputs['dat'])
# --- Define loss object
sce = losses.SparseCategoricalCrossentropy(from_logits=True)

# --- Create loss tensor
loss = sce(y_true=inputs['lbl'], y_pred=logits)
# --- Create weights
def create_weights(lbl, pos_weight=5.0):

    wgt = tf.cast(lbl > 0, dtype='float32')
    wgt = wgt + tf.cast(lbl == 2, dtype='float') * (pos_weight - 1.0)
    return wgt
    
wgt = create_weights(inputs['lbl'])

# --- Create y_true (binarized ground-truth)
y_true = tf.cast(inputs['lbl'] == 2, dtype='uint8')

# --- Create loss
sce = losses.SparseCategoricalCrossentropy(from_logits=True)(
    y_true=y_true,
    y_pred=logits,
    sample_weight=wgt)

def calculate_dsc(y_true, y_pred, weights=None, c=1):
    """
    Method to calculate the Dice score coefficient for given class

    :params

      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on

    """    
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 

    if weights is not None:
        true = true & (weights[..., 0] != 0)
        pred = pred & (weights[..., 0] != 0)

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))
dsc = calculate_dsc(y_true=inputs['lbl'], y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'dsc': dsc})
# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(dsc, name='dsc')


### Compile and train model

In [ ]:
# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Compile model
training.compile(optimizer=optimizer)
# --- Load data into memory for faster training
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []

for x in test_valid:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())

# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)
# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh
# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh

# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-08 03:22:11 ] [====================] 100.000% : Iterating | 000081    Acc: 0.6543
Sen: 0.6383
Spe: 0.6765
PPV: 0.7317
NPV: 0.5750


In [ ]:
# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='cls'),
    validation_data=G(gen_valid, dims=2, task='cls'),
    steps_per_epoch=100, 
    epochs=50,
    validation_steps=100,
    validation_freq=5)

Epoch 1/50
100/100 [==============================] - 12s 122ms/step - loss: 0.0471 - dsc: 0.9758
Epoch 2/50
100/100 [==============================] - 12s 121ms/step - loss: 0.0323 - dsc: 0.9811
Epoch 3/50
100/100 [==============================] - 12s 121ms/step - loss: 0.0481 - dsc: 0.9789
Epoch 4/50
100/100 [==============================] - 12s 121ms/step - loss: 0.0484 - dsc: 0.9822
Epoch 5/50
100/100 [==============================] - 23s 235ms/step - loss: 0.0275 - dsc: 0.9883 - val_loss: 1.5481 - val_dsc: 0.5516
Epoch 6/50
100/100 [==============================] - 12s 120ms/step - loss: 0.0299 - dsc: 0.9910
Epoch 7/50
100/100 [==============================] - 12s 122ms/step - loss: 0.0768 - dsc: 0.9700
Epoch 8/50
100/100 [==============================] - 12s 122ms/step - loss: 0.0680 - dsc: 0.9753
Epoch 9/50
100/100 [==============================] - 12s 123ms/step - loss: 0.0447 - dsc: 0.9828
Epoch 10/50
100/100 [==============================] - 23s 232ms/step - loss: 0.0

In [ ]:
# --- Serialize a model
backbone.save('./model3.hdf5')

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []


for x in test_train:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())
# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)

# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh

# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

[ 2022-06-08 03:23:32 ] [====================] 100.000% : Iterating | 000321    Acc: 0.9938
Sen: 1.0000
Spe: 0.9877
PPV: 0.9876
NPV: 1.0000


# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* accuracy
* sensitivity
* specificity
* positive predictive value (PPV)
* negative predictive value (NPV)

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please implement a prediction reduction strategy as needed for your models.

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

preds = []
trues = []

for x in test_valid:
    
    # --- Aggregate preds
    pred = backbone.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())

# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)

### Performance

The following minimum **validation cohort** performance metrics must be met for full credit:

1. **Classification**: accuracy > 0.55
2. **Segmentation**: accuracy > 0.55
3. **Custom architecture**: accuracy > 0.60

**Bonus**: the top three overall models based on **validation cohort** accuracy will recieve a +5 point (+15%) extra credit towards the final assignment.

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRAINING                              VALIDATION
          accuracy | sens | spec | PPV |  NPV | accuracy | sens | spec | PPV |  NPV
model 1
model 2
model 3
```

As above, statistics for both training and validation should be provided.

In [ ]:
# --- Create *.csv
                              
# --- Serialize *.csv
df.to_csv('./results.csv')

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion. More detailed information and tips can be found here: https://github.com/peterchang77/dl_tutor/blob/master/cs190/spring_2021/notebooks/midterm/checklist.md.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? How did you choose the network architecture implemented in your final model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.